# 🧭 NYC Yellow Taxi Data Analysis with Trino and SQL

This notebook demonstrates how to query the **NYC Yellow Taxi dataset** using **JupySQL** (the `%sql` and `%%sql`) with a Trino backend, then perform exploratory data analysis (EDA) using **Pandas** and **Altair**.

The workflow includes:
1. Loading SQL Magic and connecting to a Trino cluster
2. Inspecting schema and available tables
3. Sampling and querying data efficiently
4. Performing time-based feature engineering
5. Visualizing relationships and trends using Altair

## 🛠️ 1. Setup and Imports

We import the required libraries for data manipulation and visualization.

In [ ]:
import pandas as pd
import altair as alt
%load_ext sql
# Disable CDN loading, embed JS libraries inside the notebook
alt.renderers.enable('default')

## 🔌 2. Connect to Trino using SQL Magic

We connect to the Trino cluster using a SQLAlchemy-compliant connection URI through SQL Magic.

In [ ]:
%sql trino://trino@trino-default.okdp.sandbox:443/lakehouse/nyc_tripdata?http_scheme=https&verify=false

## 🧱 3. Inspect Table Schema

We use `DESCRIBE` to explore the `yellow` taxi table.

In [ ]:
%%sql
DESCRIBE lakehouse.nyc_tripdata.yellow

## 📋 4. List Available Tables

In [ ]:
%%sql
SHOW TABLES
FROM
  lakehouse.nyc_tripdata

## 🔍 5. Query a Random Sample (Daily Balanced Sampling)

We use a **daily random sampling** strategy with a SQL window function to ensure representation from every day across multiple months.  
Instead of `TABLESAMPLE`, this approach provides more uniform coverage and reproducibility for analysis.

We extract up to **100 random trips per day** across the first three months of 2025, capped at 3,000 total rows to keep the query lightweight.

In [ ]:
%%sql
WITH
  daily_sample AS (
    SELECT
      *,
      ROW_NUMBER() OVER (
        PARTITION BY
          DATE (tpep_pickup_datetime)
        ORDER BY
          rand ()
      ) AS rn
    FROM
      lakehouse.nyc_tripdata.yellow
    WHERE
      MONTH IN ('2025-01', '2025-02', '2025-03')
  )
SELECT
  *
FROM
  daily_sample
WHERE
  rn <= 100
LIMIT
  3000

✅ **Notes:**
- Uses `ROW_NUMBER()` and `rand()` to ensure **random yet balanced daily examples**.  
- The `month` filter restricts data to **Q1 2025** for seasonal exploration.  
- The `LIMIT` cap ensures fast execution even on large datasets.  

💡 **Tip:**  
- You can adjust `rn <= 100` to control how many random records per day are included (e.g., `rn <= 10` for faster, smaller sampling).

In [ ]:
df = _.DataFrame()
df.head()

## 🕒 6. Time-Based Feature Engineering

Extract hour of day and day of week for further analysis.

In [ ]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['hour'] = df['tpep_pickup_datetime'].dt.hour
df['day'] = df['tpep_pickup_datetime'].dt.day_name()
df[['tpep_pickup_datetime','hour','day']].head()

In [ ]:
df['day'].value_counts()

## 🧹 7. Data Quality Check
We check for problematic passenger count values.

In [ ]:
df.query("passenger_count == 0")[['trip_distance','fare_amount','total_amount']].describe()

In [ ]:
df = df[df['passenger_count'] > 0]

## 📊 8. Visualize Fare vs Distance

We create a scatterplot using **Altair**, mapping:
- X-axis → trip distance  
- Y-axis → fare amount  
- Color → passenger count  

Each point represents one taxi trip.

In [ ]:
highlight = alt.selection_point(fields=['passenger_count'], bind='legend')

chart = (
    alt.Chart(df)
    .mark_circle(size=40)
    .encode(
        x='trip_distance:Q',
        y='fare_amount:Q',
        color=alt.condition(
            highlight,
            alt.Color('passenger_count:O', scale=alt.Scale(scheme='tableau10')),
            alt.value('lightgray')
        ),
        tooltip=['tpep_pickup_datetime', 'trip_distance', 'fare_amount', 'passenger_count']
    )
    .add_params(highlight)
    .properties(title='NYC Yellow Taxi — Interactive Highlight by Passenger Count')
)
chart

💡 **Interaction:**
- Click a legend color to highlight trips for the passenger group.

🎨 **Interpretation:**
- Fares increase roughly linearly with distance.
- Outliers may indicate fixed-fare routes (e.g., airport trips).
- Dense regions near zero could reflect short-distance rides.

## ⏰ 9. Trips by Hour of Day

In [ ]:
hourly = (
    df.groupby('hour')['trip_distance']
    .count()
    .reset_index(name='trip_count')
)

alt.Chart(hourly).mark_bar(color="#4C78A8").encode(
    x=alt.X('hour:O', title='Hour of Day'),
    y=alt.Y('trip_count:Q', title='Number of Trips'),
    tooltip=[
        alt.Tooltip('hour:O', title='Hour of Day'),
        alt.Tooltip('trip_count:Q', title='Trips')
    ]
).properties(
    title='NYC Trips by Hour of Day'
)

## 📅 10. Average Fare by Day of Week

In [ ]:
daily = (
    df.groupby('day')['fare_amount']
    .mean()
    .reset_index()
    .sort_values(by='fare_amount', ascending=False)
)

alt.Chart(daily).mark_bar(color="#F58518").encode(
    x=alt.X('day:N', sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']),
    y=alt.Y('fare_amount:Q', title='Average Fare ($)'),
    tooltip=[
        alt.Tooltip('day:N', title='Day of Week'),
        alt.Tooltip('fare_amount:Q', title='Average Fare ($)', format='.2f')
    ]
).properties(
    title='Average NYC Taxi Fare by Day of Week'
)

## 🎯 11. Random Sample (BERNOULLI 1%) for Visualization
We also demonstrate %sql sampling using TABLESAMPLE BERNOULLI (1).

In [ ]:
%%sql
SELECT
  trip_distance,
  fare_amount,
  total_amount,
  passenger_count
FROM
  lakehouse.nyc_tripdata.yellow TABLESAMPLE BERNOULLI (1)
WHERE
  MONTH IN ('2025-01', '2025-02', '2025-03')
LIMIT
  3000

In [ ]:
highlight = alt.selection_point(fields=['passenger_count'], bind='legend')

chart = (
    alt.Chart(df)
    .mark_circle(size=40, opacity=0.7)
    .encode(
        x='trip_distance:Q',
        y='fare_amount:Q',
        color=alt.condition(
            highlight,
            alt.Color('passenger_count:O', scale=alt.Scale(scheme='tableau10')),
            alt.value('lightgray')
        ),
        tooltip=[
            'trip_distance',
            'fare_amount',
            'total_amount',
            'passenger_count'
        ]
    )
    .add_params(highlight)
    .properties(
        title='NYC Yellow Taxi — Fare vs Distance (Interactive Sample Highlight)'
    )
)

chart

💡 **Interaction:**
- Click a legend color to highlight trips for the passenger group.

🎨 **Interpretation:**
- Fares increase roughly linearly with distance.
- Outliers may indicate fixed-fare routes (e.g., airport trips).
- Dense regions near zero could reflect short-distance rides.

## 🚕 12. Top Pickup-Dropoff Pairs

In [ ]:
%%sql
SELECT
  pulocationid,
  dolocationid,
  COUNT(*) AS trips
FROM
  lakehouse.nyc_tripdata.yellow
WHERE
  MONTH = '2025-03'
GROUP BY
  1,
  2
ORDER BY
  trips DESC
LIMIT
  20

## 🧾 13. Statistical Summary

In [ ]:
df.describe()

## ⚙️ 14. Parameterized Query Example

In [ ]:
%config SqlMagic.named_parameters = 'enabled'

In [ ]:
month = '2025-03'
%sql SELECT COUNT(*) FROM lakehouse.nyc_tripdata.yellow WHERE month = :month

## ✅ 15. Summary
- Connected securely to **Trino** using SQL Magic  
- Queried and sampled NYC taxi data efficiently  
- Engineered time-based features  
- Built insightful visualizations using Altair